In [44]:
import numpy as np
import pandas as pd

# Data historical return for PCAR, ALTO, IKAN
returns = {
    "PCAR": [0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.76,0.00,-0.43,0.00,0.00,0.00,0.00,0.00,0.26,0.14,0.00,0.21,0.60,0.07],
    "ALTO": [0.00,0.19,-0.26,-0.21,-0.09,0.00,2.40,0.47,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.24,0.43],
    "IKAN": [-0.06,0.24,-0.55,-0.32,2.46,0.11,0.00,0.00,0.00,0.00,0.04,-0.02,0.00,0.06,0.00,0.06,0.05,0.02,0.00,0.03,0.00,-0.06,0.07,0.05]
}
average = {
    "PCAR": 0.06704,
    "ALTO": 0.17326,
    "IKAN": 0.09076
}

# Convert the data into a pandas DataFrame
df = pd.DataFrame(returns)

# Step 1: Calculate mean return (expected return) for each stock
mean_returns = df.mean()

# Step 2: Calculate covariance matrix
cov_matrix = df.cov()

# Show the results
mean_returns, cov_matrix


(PCAR    0.067083
 ALTO    0.173750
 IKAN    0.090833
 dtype: float64,
           PCAR      ALTO      IKAN
 PCAR  0.050030  0.021494 -0.003424
 ALTO  0.021494  0.310207 -0.010264
 IKAN -0.003424 -0.010264  0.276338)

In [45]:
from scipy.optimize import minimize


# Number of assets
n_assets = len(df.columns)

# Function to calculate portfolio standard deviation
def portfolio_std(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Constraint: sum of weights must be 1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Bounds for weights: between 0 and 1 (no short selling)
bounds = tuple((0, 1) for asset in range(n_assets))

# Initial guess (equal weights)
init_guess = [1/n_assets] * n_assets

# Minimize the portfolio standard deviation
opt_results = minimize(portfolio_std, init_guess, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights for minimum standard deviation
opt_weights = opt_results.x

# Calculate the expected return and standard deviation for the minimum risk portfolio
min_std = portfolio_std(opt_weights, cov_matrix)
expected_return_min_risk = np.dot(opt_weights, mean_returns)

opt_weights, min_std, expected_return_min_risk


(array([0.76574635, 0.07926968, 0.15498397]),
 0.19866455076052922,
 0.07921963517327393)

In [46]:
cov_matrix_values = np.array([
    [0.065965597, -0.008793828, -0.002786382],
    [-0.008793828, 0.031039499, -0.003421115],
    [-0.002786382, -0.003421115, 0.018121749]
])

# Convert to DataFrame
cov_matrix_df = pd.DataFrame(cov_matrix_values)

# Convert DataFrame to Series
cov_matrix_series = cov_matrix_df.stack()


portfolio_std(np.array([0.1750,	0.3361,	0.4889]), cov_matrix_df)

0.08498513034892857

In [52]:
np.dot(np.array([1/3,	1/3,	1/3]), np.array([0.07074,	0.05599,	0.03216]))

0.05296333333333333

In [30]:
cov_matrix_df

,0,1,2
0,0.065966,-0.008794,-0.002786
1,-0.008794,0.031039,-0.003421
2,-0.002786,-0.003421,0.018122
